## Customer Analysis Visualization

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import quantile_transform
from sklearn.preprocessing import minmax_scale
from sklearn.preprocessing import OneHotEncoder  ##. better to use dummy from pandas 
from sklearn.preprocessing import PowerTransformer
from scipy.stats import boxcox
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import statsmodels.api as sm
from scipy.stats import boxcox
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import mean_absolute_error as mae
pd.options.display.max_rows = 100

### Functions

In [3]:
def clean_gender(x):
    if x in ['M', 'Male']:
        return 'M'
    elif x in ["F", "female","Femal"]:
        return 'F'
    else:
        return 'U'
    
def replace_state(ca_df):
    ca_df["state"].replace({'Cali':'California', "AZ" : "Arizona", "WA":"Washington"}, inplace=True )
    return ca_df

def replace_income(ca_df):
    ca_df["income"].replace({0: np.mean(ca_df['income'])}, inplace=True )
    return ca_df

def replace_clv(ca_df): 
    ca_df["CLV"].replace({0: np.mean(ca_df['CLV']), np.nan : np.mean(ca_df['CLV'])}, inplace=True )
    return ca_df

def replace_zone(ca_df):
    ca_df["state"].replace({'California':'West Region', "Oregon" : "North West", "Washington":"East", "Arizona":"Central","Nevada":"Central"}, inplace=True )
    return ca_df

def lower_case_column_names(ca_df):
    ca_df.columns=[i.lower() for i in ca_df.columns]
    return ca_df

## Activity 1 (Monday)

- Aggregate data into one Data Frame using Pandas. Pay attention that files may have different names for the same column. therefore, make sure that you unify the columns names before concating them.
- Standardizing header names
- Deleting and rearranging columns – delete the column customer as it is only a unique identifier for each row of data
- Working with data types – Check the data types of all the columns and fix the incorrect ones (for ex. customer lifetime value and number of open complaints ). Hint: remove the percentage from the customer lifetime value and truncate it to an integer value.
- Clean the number of open complaints and extract the middle number which is changing between records. pay attention that the number of open complaints is a categorical feature.
- Filtering data and Correcting typos – Filter the data in state and gender column to standardize the texts in those columns
- Removing duplicates


## Importing data

In [4]:
file1 = pd.read_csv('datei1.csv')
file2 = pd.read_csv('datei2.csv')
file3 = pd.read_csv('datei3.csv')


In [5]:
ca_df = pd.concat([file1,file2,file3],axis = 0, sort=False)

In [6]:
ca_df = ca_df.reset_index(drop=True)

In [7]:
ca_df

,<<<<<<< HEAD,state,CLV,education,gender,income,monthly premium auto,number of open complaints,policy type,total claim amount,vehicle class
0,total claim amount,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2.704934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1131.464935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,566.472247,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,529.881344,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
17081,NaN,California,23405.987980,Bachelor,M,71941.0,73.0,0.0,Personal Auto,198.234764,Four-Door Car
17082,NaN,California,3096.511217,College,F,21604.0,79.0,0.0,Corporate Auto,379.200000,Four-Door Car
17083,NaN,California,8163.890428,Bachelor,M,0.0,85.0,3.0,Corporate Auto,790.784983,Four-Door Car
17084,NaN,California,7524.442436,College,M,21941.0,96.0,0.0,Personal Auto,691.200000,Four-Door Car


In [8]:
ca_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17086 entries, 0 to 17085
Data columns (total 11 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   <<<<<<< HEAD               4138 non-null   object 
 1   state                      7070 non-null   object 
 2   CLV                        7070 non-null   float64
 3   education                  7070 non-null   object 
 4   gender                     7070 non-null   object 
 5   income                     7070 non-null   float64
 6   monthly premium auto       7070 non-null   float64
 7   number of open complaints  7070 non-null   float64
 8   policy type                7070 non-null   object 
 9   total claim amount         7070 non-null   float64
 10  vehicle class              7070 non-null   object 
dtypes: float64(5), object(6)
memory usage: 1.4+ MB


In [9]:
ca_df['gender'].unique()

array([nan, 'M', 'F'], dtype=object)

In [10]:
ca_df.gender.value_counts(dropna=False)

NaN    10016
F       3576
M       3494
Name: gender, dtype: int64

## Using Clean_gender() Function to format gender data

In [11]:
ca_df['gender'] = list(map(clean_gender, ca_df['gender'])) 

In [12]:
ca_df.gender.value_counts(dropna=False)

U    10016
F     3576
M     3494
Name: gender, dtype: int64

## Deleting "%" from Customer Lifetime Value

In [14]:
ca_df['CLV'] = ca_df['CLV'].str.strip("%")

AttributeError: Can only use .str accessor with string values!

In [ ]:
ca_df

## Changing the CLV from categorical values to numerical values

In [ ]:
ca_df['CLV'] = pd.to_numeric(ca_df["CLV"], errors ="coerce")

In [ ]:
ca_df.info()

## Extracting the middle number of number of open complaints

In [ ]:
ca_df["number of open complaints"]= ca_df["number of open complaints"].apply(lambda x: x.split('/')[1] if type(x)==str else x)

In [ ]:
ca_df['number of open complaints'] = pd.to_numeric(ca_df['number of open complaints'],errors='coerce')

In [ ]:
ca_df

In [ ]:
ca_df.state.value_counts(dropna=False)

## Correcting typos of the states

In [ ]:
replace_state(ca_df)

In [ ]:
ca_df.state.value_counts(dropna=False)

In [ ]:
ca_df.income.value_counts(dropna=False)

## Dropping Duplicates

In [ ]:
ca_df.drop_duplicates(inplace=True)

## Activity 2 (Tuesday)

- Replacing null values – Replace missing values with means of the column (for numerical columns). Pay attention that the Income feature for instance has 0s which is equivalent to null values. (We assume here that there is no such income with 0 as it refers to missing values) Hint: numpy.nan is considered of float64 data type.
- Bucketing the data - Write a function to replace column "State" to different zones. California as West Region, Oregon as North West, and Washington as East, and Arizona and Nevada as Central

## Filling NaN cells with average income

In [ ]:
mean_income = np.mean(ca_df['income'])
ca_df['income'] = ca_df['income'].fillna(mean_income)

In [ ]:
replace_income(ca_df)

## Filling CLV NaN cells with the average CLV

In [ ]:
replace_clv(ca_df)

## Renaming states to zones

In [ ]:
ca_df.rename(columns={"state":"zone"}, inplace=True)

In [ ]:
replace_zone(ca_df)

## Activity 3 (Wednesday)

- Get the numeric data into dataframe called numerical and categorical columns in a dataframe called categoricals. (You can use np.number and np.object to select the numerical data types and categorical data types respectively)
- Now we will try to check the normality of the numerical variables visually
        - Use seaborn library to construct distribution plots for the numerical variables
        - Use Matplotlib to construct histograms
        - Do the distributions for different numerical variables look like a normal distribution
- For the numerical variables, check the multicollinearity between the features. Please note that we will use the column total_claim_amount later as the target variable.

In [ ]:
ca_df2 = pd.read_csv('Data_Marketing_Customer_Analysis_Round3.csv')

In [ ]:
ca_df2 = ca_df2.reset_index(drop=True)

In [ ]:
ca_df2

## Getting numeric data

In [ ]:
num_val = ca_df2._get_numeric_data()

In [ ]:
num_val

## Getting Categorical data

In [ ]:
cat_val = ca_df2.select_dtypes(include="object")

In [ ]:
cat_val

In [ ]:
sns.distplot(num_val, bins= 10)

In [ ]:
ca_df2.hist(figsize=(12,12))

In [ ]:
pt = PowerTransformer()

In [ ]:
val_transformed=pt.fit_transform(ca_df2._get_numeric_data().to_numpy().reshape(-1,1))
sns.displot(val_transformed)

In [ ]:
corr = ca_df2.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(9, 7))
    ax = sns.heatmap(corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)

In [ ]:
ca_df3 = pd.read_csv('Data_Marketing_Customer_Analysis_Round2.csv')

In [ ]:
ca_df3['Total Claim Amount']

In [ ]:
ca_df3.columns

In [ ]:
ca_df3.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
lower_case_column_names(ca_df3).head() # make headers uniformly lower case
ca_df3.columns

In [ ]:
plt.figure(figsize=(7,5))
#plt.subplot(2,2,1)
plt.title('Number of Responses', size='10')
plt.yticks(np.arange(0,10000,1000))
sns.countplot(ca_df2['response']) 
plt.xlabel('response',size=12)
plt.ylabel('count',size=12)

In [ ]:
plt.figure(figsize=(9,7))
plt.title('Response by sales channel', size='20')

sns.countplot(x=ca_df2['sales_channel'], hue=ca_df2['response']) 

In [ ]:
ca_df3['total claim amount'].max()

In [ ]:
pd.cut(ca_df3['total claim amount'],4)

In [ ]:
723.384+2.794

In [ ]:
pd.qcut(ca_df3['total claim amount'],[0, .25, .5, .75, 1.])

In [ ]:
plt.figure(figsize=(10,8))
plt.title('Total claim amount by sales channel', size='20')
#plt.xlabel('quartiles of total sales [2893]' )
ax = sns.countplot(x=pd.cut(ca_df3['total claim amount'],4,labels=['c1:(0,723]','c2:(723,1447]','c3:(1447,2170])','c4:(2170,2893]']), hue=ca_df3['response']) 
ax.set_xlabel('claim bracket', size=14)
plt.legend(loc='right')
plt.ylabel('count',size=14)

In [ ]:
ca_df3['income'].max()

bins = [int(99981*x+.001) for x in [0,.25,.5,.75,1]]
bins = [0,25000,50000,75000,100000]
bins

In [ ]:
income_cut = pd.cut(ca_df3['income'],bins, labels=['<25k','25k to 50k','50k to 75k','75k to 100k'],
                    duplicates='drop')#,include_lowest=True)

In [ ]:
plt.figure(figsize=(10,8))
plt.title('Response by Income Amount', size='20')
#plt.xlabel('quartiles of total sales [2893]' )
ax = sns.countplot(x=income_cut, hue=ca_df3['response']) 
ax.set_xlabel('income bracket',size=14)
ax.set_yticks(np.arange(0,2500,250))
plt.ylabel('count',size=14)

In [ ]:
numericals_df   = ca_df3.select_dtypes(include=np.number) # get numerical columns using the select_dtypes method
categoricals_df = ca_df3.select_dtypes(include=object)    # get cat columns  

In [ ]:
n_corr = num_val.corr()
mask = np.zeros_like(n_corr)
mask[np.triu_indices_from(mask)] = True
with sns.axes_style("white"):
    f, ax = plt.subplots(figsize=(10, 8))
    ax = sns.heatmap(n_corr, mask=mask,cmap='coolwarm', vmin=-1,vmax=1,annot=True, square=True)